# Решение Михаила Притугина

In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import recall_score, precision_score

In [44]:
df = pd.read_csv("train_dataset_train.csv")

In [45]:
test = pd.read_csv("test_dataset_test.csv")

In [46]:
df

,id,age_indicator,month_id,student_id,program_id,carts_created_at,spent_time_total,spent_time_to_complete_hw,completed_hw,failed_hw,...,p_total_calls,p_was_conversations,p_total_duration,support_feedback_avg,feedback_avg_d1,feedback_avg_d2,feedback_avg_d3,feedback_avg_d4,feedback_avg_d5,target
0,15182,32.0,9/30/2020,6694527,1469,8/26/2020,163.0,0.0,1.0,0.0,...,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,0
1,89385,NaN,6/30/2021,6712877,1392,8/5/2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,47931,NaN,2/28/2021,6659444,376,6/20/2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,279085,1.0,11/30/2021,7151591,1160,4/14/2021,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,7806,30.0,10/31/2020,6705666,952,7/19/2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,179932,27.0,11/30/2021,6816668,1043,10/16/2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,0
199996,257734,NaN,9/30/2021,6984939,1635,1/2/2021,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,0
199997,43549,17.0,3/31/2021,6670084,789,6/29/2020,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,NaN,0
199998,100800,NaN,6/30/2021,6917324,476,12/7/2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


## Заменим NaN`ы

В категориальных признаках на токен $other$

В числовых на число -7 (потому что оно мне нравится и не встречается в данных)

In [47]:
df["communication_type"] = df["communication_type"].fillna("other")
df["city"] = df["city"].fillna("other")
df["country"] = df["country"].fillna("other")
df["os"] = df["os"].fillna("other")
df["browser"] = df["browser"].fillna("other")
df["platform"] = df["platform"].fillna("other")
test["communication_type"] = test["communication_type"].fillna("other")
test["city"] = test["city"].fillna("other")
test["country"] = test["country"].fillna("other")
test["os"] = test["os"].fillna("other")
test["browser"] = test["browser"].fillna("other")
test["platform"] = test["platform"].fillna("other")

In [48]:
df.fillna(-7, inplace=True)
test.fillna(-7, inplace=True)

### Раздел создания новых фич

Поскольку признаки добавлялись постепенно - код выглядит немного неуклюже

In [49]:
def str2date(date):
    month, day, year = date.split("/")
    return year, f"{int(month):02d}", f"{int(day):02d}"

In [50]:
df['Year'] = df['month_id'].apply(lambda x: int(str2date(x)[0]))
df['Month'] = df['month_id'].apply(lambda x: int(str2date(x)[1]))
df['Day'] = df['month_id'].apply(lambda x: int(str2date(x)[2]))
df['CYear'] = df['carts_created_at'].apply(lambda x: int(str2date(x)[0]))
df['CMonth'] = df['carts_created_at'].apply(lambda x: int(str2date(x)[1]))
df['CDay'] = df['carts_created_at'].apply(lambda x: int(str2date(x)[2]))
test['Year'] = test['month_id'].apply(lambda x: int(str2date(x)[0]))
test['Month'] = test['month_id'].apply(lambda x: int(str2date(x)[1]))
test['Day'] = test['month_id'].apply(lambda x: int(str2date(x)[2]))
test['CYear'] = test['carts_created_at'].apply(lambda x: int(str2date(x)[0]))
test['CMonth'] = test['carts_created_at'].apply(lambda x: int(str2date(x)[1]))
test['CDay'] = test['carts_created_at'].apply(lambda x: int(str2date(x)[2]))

#### Категориальные фичи представляю как OneHotEncoding

In [51]:
unique_month = sorted(df["month_id"].unique(), key=str2date)
month2ID = {month:i for i, month in enumerate(unique_month)}
unique_carts = sorted(df["carts_created_at"].unique(), key=str2date)
carts2ID = {month:i for i, month in enumerate(unique_carts)}
unique_communication_type = df["communication_type"].unique()
communication_type2ID = {month:i for i, month in enumerate(unique_communication_type)}
unique_promo = df["promo"].unique()
promo2ID = {month:i for i, month in enumerate(unique_promo)}
unique_ABC = df["ABC"].unique()
ABC2ID = {month:i for i, month in enumerate(unique_ABC)}
unique_city = df["city"].unique()
city2ID = {month:i for i, month in enumerate(unique_city)}
unique_country = df["country"].unique()
country2ID = {month:i for i, month in enumerate(unique_country)}
unique_os = df["os"].unique()
os2ID = {month:i for i, month in enumerate(unique_os)}
unique_browser = df["browser"].unique()
browser2ID = {month:i for i, month in enumerate(unique_browser)}
unique_platform = df["platform"].unique()
platform2ID = {month:i for i, month in enumerate(unique_platform)}
unique_program_id = sorted(df["program_id"].unique())
program_id2ID = {month:i for i, month in enumerate(unique_program_id)}

In [52]:
df['month_id'] = df.month_id.map(month2ID)
test['month_id'] = test.month_id.map(month2ID)
# ------
df['carts_created_at'] = df.carts_created_at.map(carts2ID)
test['carts_created_at'] = test.carts_created_at.map(carts2ID)
# ------
df['communication_type'] = df.communication_type.map(communication_type2ID)
test['communication_type'] = test.communication_type.map(communication_type2ID)
# ------
df['promo'] = df.promo.map(promo2ID)
test['promo'] = test.promo.map(promo2ID)
# ------
df['ABC'] = df.ABC.map(ABC2ID)
test['ABC'] = test.ABC.map(ABC2ID)
# ------
df['city'] = df.city.map(city2ID)
test['city'] = test.city.map(city2ID)
# ------
df['country'] = df.country.map(country2ID)
test['country'] = test.country.map(country2ID)
# ------
df['os'] = df.os.map(os2ID)
test['os'] = test.os.map(os2ID)
# ------
df['browser'] = df.browser.map(browser2ID)
test['browser'] = test.browser.map(browser2ID)
# ------
df['platform'] = df.platform.map(platform2ID)
test['platform'] = test.platform.map(platform2ID)
# ------
df['program_id'] = df.program_id.map(program_id2ID)
test['program_id'] = test.program_id.map(program_id2ID)

In [53]:
X = df.drop(["target"], axis = 1)
y = y = np.array(df[["target"]]).ravel()

In [54]:
X

,id,age_indicator,month_id,student_id,program_id,carts_created_at,spent_time_total,spent_time_to_complete_hw,completed_hw,failed_hw,...,feedback_avg_d2,feedback_avg_d3,feedback_avg_d4,feedback_avg_d5,Year,Month,Day,CYear,CMonth,CDay
0,15182,32.0,3,6694527,147,73,163.0,0.0,1.0,0.0,...,-7.0,-7.0,-7.0,-7.0,2020,9,30,2020,8,26
1,89385,-7.0,12,6712877,140,52,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,2021,6,30,2020,8,5
2,47931,-7.0,8,6659444,20,17,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,2021,2,28,2020,6,20
3,279085,1.0,17,7151591,112,260,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,2021,11,30,2021,4,14
4,7806,30.0,4,6705666,79,46,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,2020,10,31,2020,7,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,179932,27.0,17,6816668,87,113,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,2021,11,30,2020,10,16
199996,257734,-7.0,15,6984939,174,180,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,2021,9,30,2021,1,2
199997,43549,17.0,9,6670084,60,26,-7.0,-7.0,0.0,0.0,...,-7.0,-7.0,4.0,-7.0,2021,3,31,2020,6,29
199998,100800,-7.0,12,6917324,30,154,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,2021,6,30,2020,12,7


## Обучение модели

In [34]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV

#### Создаем свою метрику для GridSearch

In [35]:
from sklearn.metrics import make_scorer

def custom_loss_function(y_true, y_pred):
    return 0.2*recall_score(y_true, y_pred, average='macro') + 0.8*precision_score(y_true, y_pred, average='macro')

custom_scorer = make_scorer(custom_loss_function, greater_is_better=True)

#### Определяем параметры поиска

In [36]:
grid = { 
    'n_estimators': [100, 300, 500],
    'max_features': ['sqrt', 'log2', 16, 32, 48, 64],
    'max_depth' : [10, 20, 30, 40, 50, 60],
    'criterion' :['entropy', 'gini'],
    'random_state' : [18]
}

In [ ]:
ex_cv = GridSearchCV(estimator=ExtraTreesClassifier(), param_grid=grid, cv=5, verbose=3, scoring=custom_scorer,
                     n_jobs=-1)
ex_cv.fit(X, y)

In [ ]:
ex_cv.best_params_

In [41]:
clf = ExtraTreesClassifier(criterion="entropy", max_depth=50, n_estimators=700, max_features=64, random_state=18)

In [112]:
clf.fit(X, y)

ExtraTreesClassifier(criterion='entropy', max_depth=50, max_features=64,
                     n_estimators=700, random_state=18)

### Предсказание на тесте и сохранение результатов

In [ ]:
test.dropna(0, inplace=True)

In [ ]:
pred = clf.predict(test)

In [124]:
sub = pd.read_csv("sample_solution.csv")

In [125]:
sub["target"] = pred

In [126]:
sub.to_csv("result.csv", index=False)

In [127]:
sub

,id,target
0,77551,0
1,227812,0
2,103035,0
3,260943,0
4,134611,0
...,...,...
84992,186427,0
84993,197918,0
84994,174961,0
84995,182226,0
